# 0. What is visit history?
* Ｇoal: 分析Chrome瀏覽器的使用歷程檔
* What-is: 瀏覽器的log（即所謂的使用歷程檔）是什麼？上面有什麼資訊？請依照以以下連結的指示，開啟Chrome瀏覽器，找到右上角的工具選項，打開「記錄」。 http://www.googlechrometutorial.com/google-chrome-general-settings/Google-chrome-web-browser-history.html
* Reference
    * [How Google Chrome stores web history](http://lowmanio.co.uk/blog/entries/how-google-chrome-stores-web-history/)
    * http://superuser.com/questions/627253/storing-browser-history-for-personal-analytics
    * http://www.v2ex.com/t/60255

#1. Copy chrome history out
* 如何獲取Chrome History檔案？
* 「access chrome history data」，之後應可找到下列網址 http://stackoverflow.com/questions/2562092/how-to-access-google-chrome-browser-history-programmatically-on-local-machine
解釋】從上面鏈結可獲知Chrome History以sqlite資料庫的檔案形態放在使用者個人電腦端（並非在雲端）

## Windows
* 如果是Windows的話是放在「C:\Documents and Settings\Local Settings\Application Data\Google\Chrome\User Data\Default」檔名為History。
* 該檔案可能是隱藏檔，所以你必須自「資料夾選項」中開啟可看見隱藏檔的選項（以下資訊參考http://www.chromium.org/user-experience/user-data-directory）。
* __Win8__:  C:\Documents and Settings\Local Settings\Application Data\Google\Chrome\User Data\Default
* __Win 7__: C:\Users\[USERNAME]\AppData\Local\Google\Chrome\User Data\Default
* __WinXP__: C:\Documents and Settings\[USERNAME]\Local Settings\Application Data\Google\Chrome\User Data\Default

##  Mac
* 若使用Mac電腦的話，History檔可能會放置在以下兩個位址：
    * /Library（資源庫）/Application Support/Google/Chrome/Default/History
    * /Users/[Your name]/Library（資源庫）/Application Support/Google/Chrome/Default/History
* Mac的History檔為隱藏檔，需要用以下的方法取消隱藏
    * 如果該檔案被隱藏無法搜尋的話，請在spotlight搜尋terminor，會開啟如下視窗（方法來源：view hidden folders on your mac using terminal０）
        * 輸入defaults write com.apple.finder AppleShowAllFiles TRUE 
        * 輸入killall Finder即可找到該資料夾
        * 之後請再輸入defaults write com.apple.finder AppleShowAllFiles FALSE 
        * 再輸入killall Finder即可恢復原狀。
    * <font color = red>或者用這方法：打開Finder後，按住Option，點選上方「前往」中的「前往資料夾」，然後直接貼上上述兩個網址之一，注意要把（資源庫）拿掉，並且把你的使用者名稱貼上。例如：rawdata/Users/jirlong/Library/Application Support/Google/Chrome/Default/History</font>

#2. Browse the History file
* Chrome history是sqlite檔而無法被一般的程式（例如NotePad）開啟。 那麼，何謂sqlite檔案？他和一般資料庫檔案（例如.mdb）有何不同？
* 無論是Firefox或Chrome history均以sqlite的資料庫格式儲存成為一個檔案，所以現在需要用一個sqlite browser來開啟所下載的history檔。
* 請上網搜尋"sqlite reader mac"或"sqlite reader windows"（Mac或Windows視你所用的系統而定，關鍵字reader可改為browser），下載並安裝。http://sourceforge.net/projects/sqlitebrowser/
* 下載該工具並且打開你所下載的History檔案。觀查History的內容，並告訴我裡面記錄了哪些資料，又以何種形式來記錄？

##2.1 Brief introduction to chrome history
* 安裝完畢後用其打開你所下載的History檔案，應可見到如下的界面。共分為三個主要部分，分別為：
    * Database Structure：用以觀看整個資料庫的綱要或欄位
    * Browse Data：用以瀏覽資料庫內存的資料<--我們最主要要看這個資料。
    * Execute SQL：允許你用這個界面操作SQL指令以查詢
    * <img src="Figures/sqlbrowser.png" style="width: 50%; height: 50%"/>
* 點選Browse Data後應可見到以下界面，開啟Table的下拉式選單，一共可發現以下資料庫的資料表：
    * meta
    * downloads：下載的檔案
    * download_url_chains
    * urls：曾經拜訪過的urls的ID與網址對照表
    * visits：主要的瀏覽歷程資料表，其中需要以ID查詢瀏覽時所拜訪的urls（查詢前項urls資料表）
    * visit_source
    * keyword_search_terms：查詢關鍵字
    * segments
    * segment_usage
* 請開啟urls資料表並開啟「檔案>export」的選項將該資料表另存為csv（comma-separated value）檔。再開啟visits資料表，並用上述的方法另存為csv檔。只要儲存為csv檔後，不僅易於程式處理，而且匯入Excel也會十分方便。

##2.2 Tables of chrome history
* History Table – urls. urls資料表主要記錄了你所曾經拜訪過的所有的網頁網址，每一筆記錄就是一筆唯一的網址，如果你曾經拜訪過某網址兩次，他也只會有一筆記錄，它記錄了以下欄位：
    * id：該網址的id，用以索引
    * url：該網址本身
    * title：該網頁的title（可能會從缺）
    * visit_count：使用者在這段時間中一共拜訪該網頁幾次
    * typed_count
    * last_visit_time：使用者最後一次拜訪該網頁的時間（你在看網頁的時候，他有時候會告訴你「您最近一次拜訪時間為...」就是看這個欄位）
    * hidden：是否為隱藏的記錄
    * favicon_id

## 2.3 SQL queries to retrieve data
* 通常類似的SQLite Database Browser都會提供輸入資料庫查詢命令的功能，點選Execute SQL後即可看到輸入SQL查詢命令的欄位。例如我們希望查詢urls這個表中，究竟有哪些網址，並且按照該網址的拜訪次數來排序的話，可以輸入「select visit_count, url from urls order by visit_count」測試之。
    * 查詢一：我希望從urls這個表格中把visit_count和url這兩個欄位的資料給選擇出來，並且按照url這個欄位進行排序。
            select visit_count, url from urls order by url 
    * 查詢二：我希望從visits這個表格中把所有的visit給找出來，並且找出url, visit_time(拜訪時間), from_visit(從哪個URL跳過來), transition(怎麼跳過來), visit_duration(拜訪多久)。且所有的結果照著id來排序。
            select url,visit_time, from_visit, transition, visit_duration from visits order by id;
    * 查詢三：我希望整合visits和urls這兩個表的資料，依據是visits表的url等於urls表的id。同時找出visits表的拜訪時間、拜訪時間長，以及這個urls表的url（因為visits表的url存的是一個編碼）。
            SELECT visits.id, visits.visit_time, visits.visit_duration, urls.url FROM visits INNER JOIN urls ON visits.url=urls.id;
* 如果我今天想要查詢資料表visits中的拜訪時間、拜訪的持續時間、拜訪的網址那要怎麼做？此時你若打開資料表visits會發現拜訪網址欄位為url，其內容是一個編號，這個id必須參考對照資料表urls的id和url欄位。因此必須要輸入以下指令：
        SELECT visits.id, visits.visit_time, visits.visit_duration, urls.url
        FROM visits
        INNER JOIN urls
        ON visits.url=urls.id;



#3. Read Chrome history by python
##3.1  import sqlite3
* 為了讀SQLite檔案，必須import sqlite3函式庫，該函式庫為Python安裝時所內建的函式庫，只需import不用另外安裝。
* 把下載下來的History檔案放在同資料中如下圖即可讀取。之後參見https://docs.python.org/2/library/sqlite3.html，
* 該文件的內容說明了sqlite3這個函式庫可以讓使用者建立table、寫入資料並讀取資料庫。

In [11]:
import sqlite3
# import urlparse
# import datetime

##3.2 Connect to history sqlite file

In [12]:
conn=sqlite3.connect("History")
# conn=sqlite3.connect("../../log_ChromeHistory/History")
cu = conn.cursor()

##3.3 執行sql查詢（Send a sql query to cu）
* w3school SQL Introduction http://www.w3schools.com/sql/sql_intro.asp
* An example: 從（from）urls這個表中，依序選出（select）visit_count與url這兩個欄位，並且照url排序（order by）
        select visit_count, url from urls order by url;

## 3.4 網址的處理 Combine urls according to their hosts
* 注意看下面query後的結果，顯然如果是mail.google.com開頭的網址，應該都算進mail.google.com就好，不要有那麼多個不一樣的網址。
        874	http://mail.google.com/
        654	https://www.facebook.com/
        500	http://www.facebook.com/
        429	https://mail.google.com/
        407	https://mail.google.com/mail/
        403	https://mail.google.com/mail/u/0/
        292	https://mail.google.com/mail/u/0/#label/%5Bntnulib%5D
        270	http://comic.sfacg.com/
        255	http://www.yahoo.com.tw/
        219	http://tw.yahoo.com/
* 所以我現在希望能夠把前面開頭是mail.google.com的都算成是拜訪同一個網址，並且累計總共拜訪幾次。
* 相當於要建一個dictionary 把網址前段mapping到出現次數的加總。
* 而現在的問題是要怎麼把網址斷開？

### 3.4.1 string operations
* 各位是否還記得我們曾經看過一個程式如下，他用sentences.split()把所有的字通通斷開了。
    1. 把所有的奇怪的字元刪除，包含
        >!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
            sentences = sentences.translate(None, string.punctuation)
    2. 把所有的字轉小寫
            sentences = sentences.lower()
    3. 把要輸入的字串依照空白斷開
            wordlist = sentences.split()
    4. 用for-each讀取wordlist中的每一個word，看他出現幾次。

### 3.4.2 split the url by "/"

### REVIEW
* __string__ split
        str1 = "jirlong hahaha"
        segs = str1.split()
* try and except
        try:
            #tried block
        except:
            # how will you handle the exception? or pass it!!
* function

###3.4.3 combine urls according to the 1st segment of url
* 下面我們將把/當成切割字串的工具，好獲得網址。
* 但是你將會遇到一個問題，就是這些資料裡面有兩個資料並非你預期中的資料，這兩個資料是因為使用者從網頁上按了「寄送信件」的按鈕後，系統自動把網頁導引到寄送信件的頁面所產生。
        mailto:jirlong@ntnu.edu.tw?bcc=96501054@nccu.edu.tw
        mailto:jackho@ntnu.edu.tw
* 總之會出錯，你有三種方法處理他
    1. 知道有這種錯誤，所以排除他！（有mailto在item[0]的通通拿掉），但問題是你怎麼知道有他？
    2. 偵測如果split()後的結果，長度大於2，代表至少有三個，那我才處理。
    3. 用try和except。
* EXPECTED RESULTS
        ERROR ITEM:mailto:jackho@ntnu.edu.tw
        ERROR ITEM:mailto:jirlong@ntnu.edu.tw?bcc=96501054@nccu.edu.tw
        [4674, u'mail.google.com']
        [3166, u'comic.sfacg.com']
        [3076, u'www.google.com']
        [1731, u'www.facebook.com']
        [1295, u'www.youtube.com']
        [907, u'moodle.ntnu.edu.tw']
        [883, u'www.google.com.tw']
        [764, u'tw.news.yahoo.com']
        [570, u'coldpic.sfacg.com']
        [551, u'docs.google.com']

In [13]:
# ## combine urls according to the 1st segment of url
# url_dict = {}
# for item in res:
#     urlseg = item[1].split('/')
#     url_dict.setdefault(urlseg[2], 0)
#     url_dict[urlseg[2]] += item[0]
# ## But you will encounter an unexpected data entry such as "mailto:jirlong@ntnu.edu.tw?bcc=96501054@nccu.edu.tw"

# ## swap the column for next step sorting (STORED TO A NEW res_list)
# res_list = []

# ## print the sorted result

##3.5 Convert the block of database query to a function
* 因為下面這兩行常常會用到，不如把它轉成一個「功能性的區塊」，就是function。
        cu.execute(sql)
        res=cu.fetchall()
* y = func(a, b) = sqrt(axa + bxb)
    * y稱為傳回值 return value
    * a, b為傳入函式的值。
    * func(a, b) = sqrt(axa + bxb) 稱為func函式的定義。
* 函式的定義大致如下
        ## Define a function
        def function_name(invalue1, invalue2, invalue3):
            result1, result2 = 0, 0
            ## Add some processing here

            return result1, result2
* 目前希望的是能夠用下面的方式進行查詢，程式碼會變得比較簡單。
        res = query("select url,visit_time, from_visit, transition, visit_duration from visits order by id ;") 
        res = query("SELECT visits.id, visits.visit_time, visits.visit_duration, urls.url FROM visits INNER JOIN urls ON visits.url=urls.id;")
* 而目前有的送出查詢的程式碼如下
        cu.execute(sql)
        res=cu.fetchall()
        print "[COMMAND(num_of_res=%d)]:%s"%(len(res), sql)      


In [14]:
## TODO: Define a query function here
def query(sql):
    cu.execute(sql)
    result=cu.fetchall()
    print "[COMMAND(num_of_res=%d)]:%s"%(len(result), sql)
    return result

In [15]:
## Call a query function
res = query("select visit_count, url from urls order by visit_count;")
print res[:1]

[COMMAND(num_of_res=14659)]:select visit_count, url from urls order by visit_count;
[(0, u'http://01-labor.blogspot.com/2011/07/django.html')]


##3.6 Query the visits table by the following sql
* 用sql browser觀察visits這個表，發現表中的url欄位儲存的不是真正的網址，而只有儲存網址的id。
* 而這個id，要對應到urls這個資料表的id欄位，才可以查出他所對應到的網址是什麼（urls中的url欄位）。所以要寫一道sql的查詢指令，"INNER JOIN"兩個表（你在這邊不需要知道INNER JOIN是什麼，只要你知道下面這道指令的意思就好了）。
        SELECT urls.url, visits.visit_time, visits.visit_duration, visits.id, visits.from_visit, visits.transition, visits.segment_id, urls.id FROM visits INNER JOIN urls ON visits.url=urls.id;
* 如果你想更瞭解SQL指令，可以參閱w3school上的說明。w3school SQL Introduction http://www.w3schools.com/sql/sql_intro.asp。

In [16]:
## TODO: Modify the following code by use of query function
# cu.execute("SELECT urls.url, visits.visit_time, visits.visit_duration FROM visits INNER JOIN urls ON visits.url=urls.id;")
# res=cu.fetchall()
res = query("SELECT urls.url, visits.visit_time, visits.visit_duration FROM visits INNER JOIN urls ON visits.url=urls.id;")
for url, t, d in res[:10]:
    segments = url.split('/')
    print t, d, segments[2]


[COMMAND(num_of_res=26346)]:SELECT urls.url, visits.visit_time, visits.visit_duration FROM visits INNER JOIN urls ON visits.url=urls.id;
13024375992405427 4763264 www.google.com.tw
13024375995839026 0 mail.google.com
13024375995839026 0 mail.google.com
13024375996831155 0 mail.google.com
13024375996831155 0 accounts.google.com
13024376003875352 118716769 accounts.google.com
13024376004883494 0 mail.google.com
13024376004883494 0 mail.google.com
13024376004883494 0 mail.google.com
13024376006908314 0 mail.google.com


##3.7 Convert timestamp to readable string
* 針對這樣的結果，看似有一些手續要處理，第一個，取出網址的前段，第二個，轉換時間成為python讀得懂的格式，不然根本看不懂。
* 這個看似整數的時間是網路處理時間的方法，為了讓python讀得懂這串數字是什麼，可以使用人家已經寫好的時間函式，把整數timestamp轉換為python datetime的方法可查詢"convert timestamp to datetime python"
    * 獲得以下結果http://stackoverflow.com/questions/9744775/how-to-convert-integer-timestamp-to-python-datetime
            >>> import datetime
            >>> your_timestamp = 1331856000000
            >>> date = datetime.datetime.fromtimestamp(your_timestamp / 1e3)
* 因為該整數是以microsecond來累計，所以要除以1000，標準的做法是除以1e3。
* duration也要轉，

## Report as HTML table
* HTML TABLES http://www.w3schools.com/html/html_tables.asp
* STEPS
    1. open an html file and write data into html tables
    2. using nbviewer [magic display to preview the html results](http://nbviewer.ipython.org/github/ipython/ipython/blob/1.x/examples/notebooks/Part%205%20-%20Rich%20Display%20System.ipynb)
            from IPython.display import HTML
            HTML('<iframe src=http://en.mobile.wikipedia.org/?useformat=mobile width=700 height=350></iframe>')
    3. Apply [CSS style](http://www.w3schools.com/html/tryit.asp?filename=tryhtml_table_headings_left)
            <head>
            <style>
            table, th, td {
                border: 1px solid black;
                border-collapse: collapse;
            }
            th, td {
                padding: 5px;
            }
            th {
                text-align: left;
            }
            </style>
</head>
    4. Apply [mouse hover (CSS Pseudo class) to change the background color of cell](http://stackoverflow.com/questions/19794433/change-font-color-and-background-in-html-on-mouseover)
            td:hover
            {
               background-color:white
            }

In [20]:
import urlparse
import datetime


epoch_start = datetime.datetime(1601,1,1) + datetime.timedelta(hours = 8)
fhtml = open("table.html", 'w')
fhtml.write(''' <head><link rel=stylesheet type="text/css" href="table.css"></head>
 <body><table>''')
for url, t, d in res[:20]:
    fhtml.write("<tr>")
    urlhost = urlparse.urlparse(url).netloc # retrieve the first segment of url
    dt = datetime.timedelta(microseconds = t) ## timestamp
    duration = datetime.timedelta(microseconds = d) ## duration
    fhtml.write("<td>%s</td>"%(epoch_start + dt))
    fhtml.write("<td>%s</td>"%(duration))
    fhtml.write("<td>%s</td>"%(urlhost))
    fhtml.write("</tr>")
fhtml.write("</table></body>")
fhtml.close()
from IPython.display import HTML
HTML('<iframe src=table.html width=1024 height=568></iframe>')

##3.8 getTime() convert timestamp to python datatime
* EXPECTED OUTPUT
        2013-09-23 10:13:12.405427 0:00:04.763264 https://www.google.com.tw/search?q=ma&oq=ma&aqs=chrome..69i57j69i65j0j5j0.361j0&sourceid=chrome&ie=UTF-8
        2013-09-23 10:13:15.839026 0:00:00 http://mail.google.com/
        2013-09-23 10:13:15.839026 0:00:00 https://mail.google.com/
        2013-09-23 10:13:16.831155 0:00:00 https://mail.google.com/mail/
        2013-09-23 10:13:16.831155 0:00:00 https://accounts.google.com/ServiceLogin?service=mail&passive=true&rm=false&continue=https://mail.google.com/mail/&ss=1&scc=1&ltmpl=default&ltmplcache=2&emr=1

In [35]:
def getTime(t, prop="duration"): # t:timestamp, prop:properties
    epoch_start = datetime.datetime(1601,1,1) + datetime.timedelta(hours = 8)
    dt = datetime.timedelta(microseconds = t)
    if prop == "time":
        return epoch_start + dt
    return dt

for url, t1, d1 in res[:10]:
    print getTime(t1, "time").strftime('%Y/%m/%d %H:%M:%S'), getTime(d1).seconds, urlparse.urlparse(url).netloc
    
## Create a getTime() Convert the timestamp and duration
# getTime(t, 'time') --> convert to timestamp
# getTime(t, 'duration') --> convert to duration
# timestamp = epoch_start + dt
# duration = dt
# urlhost = urlparse.urlparse(url).netloc

2013/09/23 10:13:12 4 www.google.com.tw
2013/09/23 10:13:15 0 mail.google.com
2013/09/23 10:13:15 0 mail.google.com
2013/09/23 10:13:16 0 mail.google.com
2013/09/23 10:13:16 0 accounts.google.com
2013/09/23 10:13:23 118 accounts.google.com
2013/09/23 10:13:24 0 mail.google.com
2013/09/23 10:13:24 0 mail.google.com
2013/09/23 10:13:24 0 mail.google.com
2013/09/23 10:13:26 0 mail.google.com


##3.9 parsed_res = parsed_res(res)
* 原本每一筆資料為[str(url), int(timestamp), int(duration)]
        "SELECT urls.url, visits.visit_time, visits.visit_duration FROM visits INNER JOIN urls ON visits.url=urls.id;"
* 我現在希望把它儲存為[str(urlhost), datetime(start_time), datetime(end_time), datetime(duration)]變成四個值。

In [42]:
parsed_res = []
for item in res: #url, timestamp, duraiton
    urlhost = urlparse.urlparse(item[0]).netloc
    start_time = getTime(item[1], "time")
    duration = getTime(item[2])
    end_time = start_time + duration
    parsed_res.append([urlhost, start_time, end_time, duration])

# function, original res to parsed res
# parsed_res = []
# for item in res:
#     urlhost = urlparse.urlparse(item[0]).netloc
#     parsed_res.append([urlhost, getTime(item[1]), getTime(item[2], "duration")])


In [41]:
## Print parsed_res in specified format or in seconds
for item in parsed_res[:30]:
    print "%s\t%s\t%s\t%s"%(item[1].strftime('%Y-%m-%d %H:%M:%S'), 
                            item[2].strftime('%Y-%m-%d %H:%M:%S'), 
                            item[3].seconds,
                            item[0])

2013-09-23 10:13:12	2013-09-23 10:13:17	4	www.google.com.tw
2013-09-23 10:13:15	2013-09-23 10:13:15	0	mail.google.com
2013-09-23 10:13:15	2013-09-23 10:13:15	0	mail.google.com
2013-09-23 10:13:16	2013-09-23 10:13:16	0	mail.google.com
2013-09-23 10:13:16	2013-09-23 10:13:16	0	accounts.google.com
2013-09-23 10:13:23	2013-09-23 10:15:22	118	accounts.google.com
2013-09-23 10:13:24	2013-09-23 10:13:24	0	mail.google.com
2013-09-23 10:13:24	2013-09-23 10:13:24	0	mail.google.com
2013-09-23 10:13:24	2013-09-23 10:13:24	0	mail.google.com
2013-09-23 10:13:26	2013-09-23 10:13:26	0	mail.google.com
2013-09-23 10:13:31	2013-09-23 10:13:31	0	mail.google.com
2013-09-23 10:13:34	2013-09-23 10:13:34	0	mail.google.com
2013-09-23 10:13:42	2013-09-23 10:13:42	0	mail.google.com
2013-09-23 10:13:43	2013-09-23 10:13:43	0	talkgadget.google.com
2013-09-23 10:13:43	2013-09-23 10:13:43	0	talkgadget.google.com
2013-09-23 10:13:44	2013-09-23 10:13:44	0	mail.google.com
2013-09-23 10:13:54	2013-09-23 10:13:54	0	mail.g

##3.10 Merge adjacent visits
* 由於前述的結果，有太多mail或者talkgadget是相同的服務，我希望前後如果一樣的話，就把他銜接起來。

In [54]:
print "VISITS BEFORE MERGE:%d"%(len(parsed_res))
merged_visits = []
uniq_url = None
for visit in parsed_res: #urlhost, stime, etime, duration
    if uniq_url != visit[0]:
        uniq_url = visit[0]
        merged_visits.append([uniq_url, visit[1], visit[2], visit[3]])
    else:
        if visit[1] - merged_visits[-1][2] > getTime(300000000):
            merged_visits.append([uniq_url, visit[1], visit[2], visit[3]])
        else:
            try:
                merged_visits[-1][2] = visit[2]
                merged_visits[-1][3] = visit[2] - merged_visits[-1][1]
            except:
                print merged_visits[-1]
print "VISITS BEFORE MERGE:%d"%(len(merged_visits))


VISITS BEFORE MERGE:26346
VISITS BEFORE MERGE:8406


In [51]:
print "VISITS BEFORE MERGE:%d"%(len(parsed_res))
merged_visits = []
uniq_url = None
for visit in parsed_res: #urlhost, stime, etime, duration
    if uniq_url != visit[0]:
        uniq_url = visit[0]
        merged_visits.append([uniq_url, visit[1], visit[2], visit[3]])
    else:
        try:
            merged_visits[-1][2] = visit[2]
            merged_visits[-1][3] = visit[2] - merged_visits[-1][1]
        except:
            print merged_visits[-1]
print "VISITS BEFORE MERGE:%d"%(len(merged_visits))


VISITS BEFORE MERGE:26346
VISITS BEFORE MERGE:6656


In [55]:
for v in merged_visits[:20]:
    print v[1], v[3].seconds, v[0]

2013-09-23 10:13:12.405427 4 www.google.com.tw
2013-09-23 10:13:15.839026 0 mail.google.com
2013-09-23 10:13:16.831155 125 accounts.google.com
2013-09-23 10:13:24.883494 17 mail.google.com
2013-09-23 10:13:43.689719 0 talkgadget.google.com
2013-09-23 10:13:44.907833 9 mail.google.com
2013-09-23 10:14:14.229230 53 talkgadget.google.com
2013-09-23 10:15:22.592121 40 mail.google.com
2013-09-23 10:16:07.567594 0 talkgadget.google.com
2013-09-23 10:16:07.645335 0 mail.google.com
2013-09-23 10:16:17.929707 0 talkgadget.google.com
2013-09-23 10:16:26.780083 21 mail.google.com
2013-09-23 10:16:48.399798 0 talkgadget.google.com
2013-09-23 10:16:57.364934 5 mail.google.com
2013-09-23 10:25:28.395769 116 www.google.com.tw
2013-09-23 10:25:34.618732 0 maps.google.com.tw
2013-09-23 10:26:36.084441 46 www.mapquest.com
2013-09-23 10:30:14.557195 1064 mail.google.com
2013-09-23 10:30:28.572185 0 talkgadget.google.com
2013-09-23 10:30:39.801317 5 www.google.com.tw


##3.10.1 Filter in neccesary entry
* 有了merged_visits後，我現在希望的是只留下某些我所要觀察的項目，例如我只想觀察使用者使用facebook、mail、youtube的情形，那要怎麼抓出來？
* 所以我現在要把merged_visits轉成filtered_visits。

In [58]:
filtered_visits = []
filter_in = ['dropbox', 'dictionary', 'facebook', 'mail', 'youtube', 'comic', 'dm5', 'mobile01']
for row in merged_visits:
#     counter = 0
#     for word in filter_in:
#         if word in row[0]:
#             counter += 1
#     if counter > 0:
#         filtered_visits.append(row)
    if len([word for word in filter_in if word in row[0]])>0: ## LIST COMPREHENSION
        filtered_visits.append(row)
# for item in inlist:
#     if len([x for x in filter_in if x in item[0]])>0:
#         res.append(item)

In [61]:
print len(filtered_visits)
print "%s\t%-14s\t%s\t%-10s"%('date', 'time', 'duration', 'url')

## how to print date only?
for r in filtered_visits[:30]:
    print "%s\t%-14s\t%s\t%-10s"%(r[1], r[2],  r[3], r[0])

2931
date	time          	duration	url       
2013-09-23 10:13:15.839026	2013-09-23 10:13:16.831155	0:00:00.992129	mail.google.com
2013-09-23 10:13:24.883494	2013-09-23 10:13:42.387513	0:00:17.504019	mail.google.com
2013-09-23 10:13:44.907833	2013-09-23 10:13:54.310997	0:00:09.403164	mail.google.com
2013-09-23 10:15:22.592121	2013-09-23 10:16:02.620901	0:00:40.028780	mail.google.com
2013-09-23 10:16:07.645335	2013-09-23 10:16:07.645335	0:00:00	mail.google.com
2013-09-23 10:16:26.780083	2013-09-23 10:16:48.154514	0:00:21.374431	mail.google.com
2013-09-23 10:16:57.364934	2013-09-23 10:17:02.501640	0:00:05.136706	mail.google.com
2013-09-23 10:30:14.557195	2013-09-23 10:47:59.389435	0:17:44.832240	mail.google.com
2013-09-23 10:50:50.933290	2013-09-23 11:00:13.362823	0:09:22.429533	mail.google.com
2013-09-23 11:13:08.854806	2013-09-23 11:21:21.548131	0:08:12.693325	mail.google.com
2013-09-23 11:13:24.737812	2013-09-23 11:13:54.289632	0:00:29.551820	www.facebook.com
2013-09-23 11:20:54.024315

### First try of using Pandas
* Tutorials http://cloga.info/python/%E6%95%B0%E6%8D%AE%E7%A7%91%E5%AD%A6/2013/09/17/pandas_intro/

In [63]:
import pandas as pd
# df = pd.DataFrame(filtered_visits)
df = pd.DataFrame(merged_visits) ## convert list to pandas dataframe format
df.head() ## print the first 5 data for example

0                          1  \
0      www.google.com.tw 2013-09-23 10:13:12.405427   
1        mail.google.com 2013-09-23 10:13:15.839026   
2    accounts.google.com 2013-09-23 10:13:16.831155   
3        mail.google.com 2013-09-23 10:13:24.883494   
4  talkgadget.google.com 2013-09-23 10:13:43.689719   

                           2               3  
0 2013-09-23 10:13:17.168691 00:00:04.763264  
1 2013-09-23 10:13:16.831155 00:00:00.992129  
2 2013-09-23 10:15:22.592121 00:02:05.760966  
3 2013-09-23 10:13:42.387513 00:00:17.504019  
4 2013-09-23 10:13:43.689719        00:00:00

In [65]:
df.describe()

0                           1  \
count              8406                        8406   
unique             1077                        7538   
top     mail.google.com  2013-10-29 15:28:08.447525   
freq               1201                           4   
first               NaN  2013-09-12 23:40:57.320920   
last                NaN  2013-12-11 17:50:13.645286   
mean                NaN                         NaN   
std                 NaN                         NaN   
min                 NaN                         NaN   
25%                 NaN                         NaN   
50%                 NaN                         NaN   
75%                 NaN                         NaN   
max                 NaN                         NaN   

                                 2                         3  
count                         8406                      8406  
unique                        7885                       NaN  
top     2013-10-14 11:06:56.260744                       NaN  
freq                             4                       NaN  
first   2013-09-12 23:40:57.320920                       NaN  
last    2013-12-11 17:51:33.883506                       NaN  
mean                           NaN    0 days 00:04:54.662657  
std                            NaN    0 days 01:44:07.732084  
min                            NaN  -3 days +08:44:32.568924  
25%                            NaN           0 days 00:00:00  
50%                            NaN    0 days 00:00:03.519585  
75%                            NaN    0 days 00:01:08.759183  
max                            NaN    2 days 21:34:15.925756

##3.11 Convert the time to specified string format

In [67]:
# print filtered_visits[0]
for r in filtered_visits[:20]:
#     print "%s\t%s\t%s\t%s"%(r[0], r[1].isoformat(), r[2].isoformat(), r[3].seconds)
    print "%s\t%s\t%s\t%s"%(r[1].strftime('%Y-%m-%d %H:%M:%S'), r[2].strftime('%Y-%m-%d %H:%M:%S'), r[3].seconds, r[0])

2013-09-23 10:13:15	2013-09-23 10:13:16	0	mail.google.com
2013-09-23 10:13:24	2013-09-23 10:13:42	17	mail.google.com
2013-09-23 10:13:44	2013-09-23 10:13:54	9	mail.google.com
2013-09-23 10:15:22	2013-09-23 10:16:02	40	mail.google.com
2013-09-23 10:16:07	2013-09-23 10:16:07	0	mail.google.com
2013-09-23 10:16:26	2013-09-23 10:16:48	21	mail.google.com
2013-09-23 10:16:57	2013-09-23 10:17:02	5	mail.google.com
2013-09-23 10:30:14	2013-09-23 10:47:59	1064	mail.google.com
2013-09-23 10:50:50	2013-09-23 11:00:13	562	mail.google.com
2013-09-23 11:13:08	2013-09-23 11:21:21	492	mail.google.com
2013-09-23 11:13:24	2013-09-23 11:13:54	29	www.facebook.com
2013-09-23 11:20:54	2013-09-23 11:20:54	0	www.facebook.com
2013-09-23 11:43:11	2013-09-23 12:39:16	3364	tw.dictionary.yahoo.com
2013-09-23 12:03:45	2013-09-23 12:03:55	10	mail.google.com
2013-09-23 12:04:18	2013-09-23 12:05:38	79	mail.google.com
2013-09-23 12:17:03	2013-09-23 12:20:19	195	mail.google.com
2013-09-23 12:21:43	2013-09-23 12:23:32	109	

In [6]:
## Save the result to a filtered_table.html and display it!!
# ftable = file("visit_table.html", 'w')
# ## "WRITE SOMETHING INTO FILE"
# ftable.write("WRITE SOMETHING INTO FILE")
# ftable.close()
# HTML('<iframe src=visit_table.html width=700 height=350></iframe>')

## Report timeline

In [69]:
def report(vlist, fname):
    fout=file(fname, 'w')
    fout.write('''
<script type="text/javascript" src="https://www.google.com/jsapi?autoload={'modules':[{'name':'visualization',
       'version':'1','packages':['timeline']}]}"></script>
<script type="text/javascript">

google.setOnLoadCallback(drawChart);
function drawChart() {

  var container = document.getElementById('example5.1');
  var chart = new google.visualization.Timeline(container);
  var dataTable = new google.visualization.DataTable();
  dataTable.addColumn({ type: 'string', id: 'Room' });
  dataTable.addColumn({ type: 'string', id: 'Name' });
  dataTable.addColumn({ type: 'date', id: 'Start' });
  dataTable.addColumn({ type: 'date', id: 'End' });
  dataTable.addRows([
    ''')
    for v in vlist:
        fout.write('["%s", "%s", new Date(0,0,0,%d,%d,%d), new Date(0,0,0,%d,%d,%d)],\n'%(v[1].strftime('%Y-%m-%d'), v[0], v[1].hour, v[1].minute, v[1].second, v[2].hour, v[2].minute, v[2].second))
    fout.write('''
        ]);
  var options = {
      colors:['#33ccff', '#ff66cc', '#339933','#ffcc33','#ff0000', '#333333', '#996600'],
    timeline: { rowLabelStyle: {fontName: 'Helvetica', fontSize: 24, color: '#603913' },
                barLabelStyle: { fontName: 'Garamond', fontSize: 14 } }
  };
  chart.draw(dataTable, options);
}
</script>
<div id="example5.1" style="width: 3900px; height: 1000px;"></div>
    ''')
    fout.close()


In [73]:
report(filtered_visits[120:320], "viz.html")
HTML('<iframe src=viz.html width=700 height=350></iframe>')

# Reference